In [1]:
%load_ext nb_black
%load_ext autoreload
%autoreload 2

import os

print(os.getcwd())


def update_working_directory():
    from pathlib import Path

    p = Path(os.getcwd()).parents[0]
    os.chdir(p)
    print(p)


update_working_directory()

/Users/admin/Projects/vocabulary_learning/notebooks
/Users/admin/Projects/vocabulary_learning


<IPython.core.display.Javascript object>

In [2]:
import numpy as np
import pandas as pd

from src.data.collect_data import *

<IPython.core.display.Javascript object>

In [4]:
vocab = initiate_vocab(test=False)

historical_data, tries_session = initiate_session()

nb_known_words_session = 0

<IPython.core.display.Javascript object>

In [5]:
for i in range(100):

    # Choose language
    input_language, output_language = choose_a_language(vocab)
    if input_language == -1:
        break

    # Pick a random word
    id_vocab = choose_a_word(vocab, input_language, output_language)

    # Take a guess
    your_guess = prompt_question(
        vocab,
        id_vocab,
        input_language,
        output_language,
        nb_known_words_session,
        tries_session,
    )

    # Answer analysis
    is_it_correct = (
        your_guess
        == vocab.loc[vocab["id_vocab"] == id_vocab, output_language].values[0]
    )
    is_it_another_word = your_guess in vocab[output_language].values

    # Do we want to stop?
    if your_guess == "STOP IT":
        print(
            "Your last word was '{}={}'".format(
                vocab.loc[a, input_language].values[0],
                vocab.loc[vocab["id_vocab"] == id_vocab, output_language].values[0],
            )
        )
        break

    # If correct answer
    if is_it_correct:
        if_correct()
        nb_known_words_session += 1
        write_it_again = None

    # If wrong answer
    if not is_it_correct:
        write_it_again = if_not_correct(
            vocab,
            id_vocab,
            input_language,
            output_language,
            your_guess,
            is_it_another_word,
        )

    # Add to historical data
    historical_data = add_historical_data(
        historical_data,
        vocab,
        id_vocab,
        input_language,
        output_language,
        is_it_correct,
        your_guess,
        write_it_again,
    )

    # Update vocab
    vocab = update_vocab(
        vocab, id_vocab, input_language, output_language, is_it_correct
    )

    tries_session += 1

0/0 - What is the english word for 'der Dom'?   the cathedral
Correct!
1/1 - What is the german word for 'the red card'?   die rote Carte
Sorry, it was 'the red card = die rote Karte'
Write it again   die rote Karte the red card die rote Karte
1/2 - What is the english word for 'orange'?   orange
Correct!
2/3 - What is the german word for 'the piece'?   die Stück
Sorry, it was 'the piece = das Stück'
Write it again   das Stück the piece das Stück
2/4 - What is the english word for 'mag'?   like
Correct!
3/5 - What is the german word for 'the soup'?   die Suppe
Correct!
4/6 - What is the german word for 'the ritual'?   das Ritual
Correct!
5/7 - What is the english word for 'das Hotel'?   the hotel
Correct!
6/8 - What is the english word for 'der Blumenkohl'?   the flower
Sorry, it was 'der Blumenkohl = the cauliflower'
You confused it with 'the flower = die Blume'
Write it again   der Blumenkohl the cauliflower der Blumenkohl
6/9 - What is the german word for 'the chair'?   der Stuhl
Co

58/79 - What is the english word for 'das Bett'?   the bed
Correct!
59/80 - What is the german word for 'to kill'?   killen
Correct!
60/81 - What is the german word for 'the stress'?   das Stress
Sorry, it was 'the stress = der Stress'
Write it again   der Stress the stress der Stress
60/82 - What is the english word for 'heute'?   today
Correct!
61/83 - What is the german word for 'the sea'?   der See
Sorry, it was 'the sea = das Meer'
You confused it with 'der See = the lake'
Write it again   das Meer the seas das Meer
61/84 - What is the german word for 'November'?   November
Correct!
62/85 - What is the english word for 'das Wort'?   the word
Correct!
63/86 - What is the english word for 'lecker'?   delicious
Correct!
64/87 - What is the english word for 'der Freund'?   the boyfriend
Correct!
65/88 - What is the english word for 'Mai'?   May
Correct!
66/89 - What is the english word for 'hübsch'?   pretty
Correct!
67/90 - What is the german word for 'Wednesday'?   Mittwoc
Sorry, it

<IPython.core.display.Javascript object>

In [6]:
# Update and save historical data
update_historical_data(historical_data, test=False)

<IPython.core.display.Javascript object>

In [7]:
# Update and save vocab
finalize_vocab(vocab, test=False)

<IPython.core.display.Javascript object>